# Hyperoptimization with optuna

## 1. Dataset

Example for dataset ESOL from Moleculenet.

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
from kgcnn.data.datasets.ESOLDataset import ESOLDataset
dataset = ESOLDataset()
dataset.set_attributes()
dataset.map_list(method="set_edge_indices_reverse")

ERROR:kgcnn.mol.convert:Can not import `RDKit` package for conversion.
INFO:kgcnn.data.download:Checking and possibly downloading dataset with name ESOL
INFO:kgcnn.data.download:Dataset directory located at C:\Users\patri\.kgcnn\datasets
INFO:kgcnn.data.download:Dataset directory found. Done.
INFO:kgcnn.data.download:Dataset found. Done.
INFO:kgcnn.data.ESOL:Found SDF C:\Users\patri\.kgcnn\datasets\ESOL\delaney-processed.sdf of pre-computed structures.
INFO:kgcnn.data.ESOL:Read molecules from mol-file.
INFO:kgcnn.data.ESOL: ... process molecules 0 from 1128
INFO:kgcnn.mol.encoder:OneHotEncoder Symbol found ['O', 'C', 'N', 'S', 'Cl', 'P', 'F', 'I', 'Br']
INFO:kgcnn.mol.encoder:OneHotEncoder Hybridization found [rdkit.Chem.rdchem.HybridizationType.SP3, rdkit.Chem.rdchem.HybridizationType.SP, rdkit.Chem.rdchem.HybridizationType.SP2]
INFO:kgcnn.mol.encoder:OneHotEncoder TotalDegree found [2, 4, 1, 3]
INFO:kgcnn.mol.encoder:OneHotEncoder TotalNumHs found [1, 2, 0, 3, 4]
INFO:kgcnn.mol.encod

<ESOLDataset [{'node_symbol': array(['O', 'C', 'C', 'O', 'C', 'O', 'C', 'C', 'O', 'C', 'O', 'C', 'C',
       'N', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'O', 'C', 'O', 'C', 'O',
       'C', 'O', 'C', 'O', 'C', 'O'], dtype='<U1'), 'node_number': array([8, 6, 6, 8, 6, 8, 6, 6, 8, 6, 8, 6, 6, 7, 6, 6, 6, 6, 6, 6, 6, 8,
       6, 8, 6, 8, 6, 8, 6, 8, 6, 8]), 'edge_indices': array([[ 0,  1],
       [ 1,  0],
       [ 1,  2],
       [ 2,  1],
       [ 2,  3],
       [ 2, 30],
       [ 3,  2],
       [ 3,  4],
       [ 4,  3],
       [ 4,  5],
       [ 4, 26],
       [ 5,  4],
       [ 5,  6],
       [ 6,  5],
       [ 6,  7],
       [ 7,  6],
       [ 7,  8],
       [ 7, 24],
       [ 8,  7],
       [ 8,  9],
       [ 9,  8],
       [ 9, 10],
       [ 9, 20],
       [10,  9],
       [10, 11],
       [11, 10],
       [11, 12],
       [11, 14],
       [12, 11],
       [12, 13],
       [13, 12],
       [14, 11],
       [14, 15],
       [14, 19],
       [15, 14],
       [15, 16],
       [16, 15],
  

## 2. Model

Pick DMPNN as example model.

In [3]:
from kgcnn.literature.DMPNN import make_model

In [4]:
model_config = {
    "name": "DMPNN",
    "inputs": [
        {"shape": [None, 41], "name": "node_attributes", "dtype": "float32", "ragged": True},
        {"shape": [None, 11], "name": "edge_attributes", "dtype": "float32", "ragged": True},
        {"shape": [None, 2], "name": "edge_indices", "dtype": "int64", "ragged": True},
        {"shape": [None, 1], "name": "edge_indices_reverse", "dtype": "int64", "ragged": True}
    ],
    "input_embedding": {"node": {"input_dim": 95, "output_dim": 64},
                        "edge": {"input_dim": 5, "output_dim": 64}},
    "pooling_args": {"pooling_method": "sum"},
    "edge_initialize": {"units": 128, "use_bias": True, "activation": "relu"},
    "edge_dense": {"units": 128, "use_bias": True, "activation": "linear"},
    "edge_activation": {"activation": "relu"},
    "node_dense": {"units": 128, "use_bias": True, "activation": "relu"},
    "verbose": 10, "depth": 5,
    "dropout": {"rate": 0.1},
    "output_embedding": "graph",
    "output_mlp": {
        "use_bias": [True, True, False], "units": [64, 32, 1],
        "activation": ["relu", "relu", "linear"]
    }
}
# Test making model
model = make_model(**model_config)

INFO:kgcnn.model.utils:Updated model kwargs:
INFO:kgcnn.model.utils:{'name': 'DMPNN', 'inputs': [{'shape': [None, 41], 'name': 'node_attributes', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 11], 'name': 'edge_attributes', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 2], 'name': 'edge_indices', 'dtype': 'int64', 'ragged': True}, {'shape': [None, 1], 'name': 'edge_indices_reverse', 'dtype': 'int64', 'ragged': True}], 'input_embedding': {'node': {'input_dim': 95, 'output_dim': 64}, 'edge': {'input_dim': 5, 'output_dim': 64}, 'graph': {'input_dim': 100, 'output_dim': 64}}, 'pooling_args': {'pooling_method': 'sum'}, 'use_graph_state': False, 'edge_initialize': {'units': 128, 'use_bias': True, 'activation': 'relu'}, 'edge_dense': {'units': 128, 'use_bias': True, 'activation': 'linear'}, 'edge_activation': {'activation': 'relu'}, 'node_dense': {'units': 128, 'use_bias': True, 'activation': 'relu'}, 'verbose': 10, 'depth': 5, 'dropout': {'rate': 0.1}, 'output_embedding': 'gr

In [5]:
dataset.clean(model_config["inputs"])

INFO:kgcnn.data.ESOL:Property 'edge_attributes' is an empty list for graph '934'.
INFO:kgcnn.data.ESOL:Property 'edge_indices' is an empty list for graph '934'.
INFO:kgcnn.data.ESOL:Property 'edge_indices_reverse' is an empty list for graph '934'.


array([934])

In [6]:
labels = np.expand_dims(dataset.get("graph_labels"), axis=-1)
labels.shape

(1127, 1)

## 3. Optuna

Loading optuna for hyperoptimization example. Example is modified from optuna [github](https://github.com/optuna/optuna-examples/blob/main/keras/keras_simple.py>) or web page https://optuna.org/#code_examples.

For this example we neglect target standardization or more metric options.

In [7]:
import warnings
import optuna
from copy import deepcopy
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from keras.backend import clear_session

In [8]:
BATCHSIZE = 128
EPOCHS = 100

In [9]:
# Define objective function
def objective(trial):
    # Clear clutter from previous Keras session graphs.
    clear_session()

    train_index, test_index = train_test_split(np.arange(len(dataset)), test_size=0.25, random_state=42)
    x_train, y_train = dataset[train_index].tensor(model_config["inputs"]), labels[train_index]
    x_valid, y_valid = dataset[test_index].tensor(model_config["inputs"]), labels[test_index]
    
    hyper_trial = deepcopy(model_config)
    hyper_trial["depth"] = trial.suggest_int("depth", 1, 6, 2)
    hyper_trial["pooling_args"]["pooling_method"] = trial.suggest_categorical("pooling_method", ["sum", "mean"])
    units = trial.suggest_int("units", 32, 256, 32)
    hyper_trial["node_dense"]["units"] = units
    hyper_trial["edge_dense"]["units"] = units
    hyper_trial["edge_initialize"]["units"] = units
    # print(hyper_trial)
    
    model = make_model(**hyper_trial)
    
    # We compile our model
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    model.compile(
        loss="mean_squared_error",
        optimizer=Adam(learning_rate=learning_rate),
        metrics=["mean_absolute_error"],
    )

    model.fit(
        x_train,
        y_train,
        validation_data=(x_valid, y_valid),
        shuffle=True,
        batch_size=BATCHSIZE,
        epochs=EPOCHS,
        verbose=False,
    )

    # Evaluate the model accuracy on the validation set.
    score = model.evaluate(x_valid, y_valid, verbose=0)
    return score[1]

### Optimization

In [10]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100, timeout=600)

[I 2023-02-09 15:54:42,344] A new study created in memory with name: no-name-32a1234d-5703-482c-ac34-3ce69941d353
C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\optuna\distributions.py:560: UserWarning: The distribution is specified by [1, 6] and step=2, but the range is not divisible by `step`. It will be replaced by [1, 5].
  warnings.warn(
INFO:kgcnn.model.utils:Updated model kwargs:
INFO:kgcnn.model.utils:{'name': 'DMPNN', 'inputs': [{'shape': [None, 41], 'name': 'node_attributes', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 11], 'name': 'edge_attributes', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 2], 'name': 'edge_indices', 'dtype': 'int64', 'ragged': True}, {'shape': [None, 1], 'name': 'edge_indices_reverse', 'dtype': 'int64', 'ragged': True}], 'input_embedding': {'node': {'input_dim': 95, 'output_dim': 64}, 'edge': {'input_dim': 5, 'output_dim': 64}, 'graph': {'input_dim': 100, 'output_dim': 64}}, 'pooling_args': {'pooling_method': 'sum'}, 

C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Reshape:0", shape=(None, 128), dtype=float32), dense_shape=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed_4/gather_nodes_outgoing_5/Reshape_1:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed_4/gather_nodes_outgoing_5/Reshape:0", shape=(None, 12

C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/pooling_local_edges_1/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/DMPNN/pooling_local_edges_1/Reshape:0", shape=(None, 32), dtype=float32), dense_shape=Tensor("gradient_tape/DMPNN/pooling_local_edges_1/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed/gather_nodes_outgoing_1/Reshape_1:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed/gather_nodes_outgoing_1/Reshape:0", shape=(None, 32), dt

[I 2023-02-09 15:56:05,239] Trial 3 finished with value: 0.7480711936950684 and parameters: {'depth': 3, 'pooling_method': 'sum', 'units': 160, 'learning_rate': 5.021049067441841e-05}. Best is trial 1 with value: 0.4369063079357147.
C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\optuna\distributions.py:560: UserWarning: The distribution is specified by [1, 6] and step=2, but the range is not divisible by `step`. It will be replaced by [1, 5].
  warnings.warn(
INFO:kgcnn.model.utils:Updated model kwargs:
INFO:kgcnn.model.utils:{'name': 'DMPNN', 'inputs': [{'shape': [None, 41], 'name': 'node_attributes', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 11], 'name': 'edge_attributes', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 2], 'name': 'edge_indices', 'dtype': 'int64', 'ragged': True}, {'shape': [None, 1], 'name': 'edge_indices_reverse', 'dtype': 'int64', 'ragged': True}], 'input_embedding': {'node': {'input_dim': 95, 'output_dim': 64}, 'edge': {'input_

C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Reshape:0", shape=(None, 64), dtype=float32), dense_shape=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed_4/gather_nodes_outgoing_5/Reshape_1:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed_4/gather_nodes_outgoing_5/Reshape:0", shape=(None, 64)

...

C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Reshape:0", shape=(None, 160), dtype=float32), dense_shape=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed_4/gather_nodes_outgoing_5/Reshape_1:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed_4/gather_nodes_outgoing_5/Reshape:0", shape=(None, 16

C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Reshape:0", shape=(None, 96), dtype=float32), dense_shape=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed_4/gather_nodes_outgoing_5/Reshape_1:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed_4/gather_nodes_outgoing_5/Reshape:0", shape=(None, 96)

C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Reshape:0", shape=(None, 128), dtype=float32), dense_shape=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed_4/gather_nodes_outgoing_5/Reshape_1:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed_4/gather_nodes_outgoing_5/Reshape:0", shape=(None, 12

C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Reshape:0", shape=(None, 96), dtype=float32), dense_shape=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed_4/gather_nodes_outgoing_5/Reshape_1:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed_4/gather_nodes_outgoing_5/Reshape:0", shape=(None, 96)

C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Reshape:0", shape=(None, 192), dtype=float32), dense_shape=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed_4/gather_nodes_outgoing_5/Reshape_1:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed_4/gather_nodes_outgoing_5/Reshape:0", shape=(None, 19

In [11]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials: 27
Best trial:
  Value: 0.4369063079357147
  Params: 
    depth: 5
    pooling_method: sum
    units: 128
    learning_rate: 0.0026645665219035054
